In [13]:
from src.data.loader import load_interactions, load_games
from src.evaluation.splitter import split_train_in_out
from src.evaluation.evaluator import evaluate_model
from src.models.ease import EASE
from src.novelty.distance import (
    build_genre_similarity_matrix,
    build_genre_distance_matrix,
)
import numpy as np
from src.pipelines.save import save_submission
from src.config import LAMBDA_REG, TOP_K, N_EVAL_USERS, SEED

# PART 1: OFFLINE EVALUATION

In [2]:
# 1. Load data
train = load_interactions(train=True)
games = load_games()

# maps item_id -> publisher (needed for publisher Gini)
publisher_mapper = games.set_index("item_id")["publisher"]

In [3]:
# 2. Local split (Codabench-like: 1 holdout per user)
train_in_full, train_out_full = split_train_in_out(train, seed=SEED)

In [4]:
# 3. Sample a subset of users for faster offline eval
all_users = train_out_full["user_id"].unique()          # only users with a holdout
rng = np.random.default_rng(SEED)
n_eval = min(N_EVAL_USERS, len(all_users))
sample_users = rng.choice(all_users, size=n_eval, replace=False)

In [5]:
train_in = train_in_full[train_in_full["user_id"].isin(sample_users)].reset_index(drop=True)
train_out = train_out_full[train_out_full["user_id"].isin(sample_users)].reset_index(drop=True)

print(f"Offline eval on {n_eval} users "
      f"(train_in rows = {len(train_in)}, train_out rows = {len(train_out)})")

Offline eval on 3000 users (train_in rows = 132505, train_out rows = 3000)


In [6]:
# 4. Build distance and similarity once (all items)
item_similarity = build_genre_similarity_matrix(games)
item_distance = build_genre_distance_matrix(games)

In [14]:
# 5. Fit EASE on *fold-in* for these users and get recommendations
model = EASE(lambda_reg=LAMBDA_REG)

# train_in = history (fold-in) for sampled users
# test_in = same df; we score these users
recs_offline = model.recommend(train_in, train_in, top_k=TOP_K)

In [15]:
# 6. Evaluate offline baseline on the sampled users
metrics_offline = evaluate_model(
    recs_offline,
    train_in,
    train_out,
    publisher_mapper=publisher_mapper,
    item_similarity=item_similarity,
    item_distance=item_distance,
)
print(f"Offline EASE baseline (lambda_reg = {LAMBDA_REG}) on {n_eval} users")
print(metrics_offline)

Offline EASE baseline (lambda_reg = 100) on 3000 users
{'ndcg': 0.22782722457729254, 'recall': 0.4196666666666667, 'user_coverage': 1.0, 'item_gini': 0.8227847473342605, 'publisher_gini': 0.8274178751009893, 'item_coverage': 0.12741449583554848, 'intra_list_similarity': 0.4565796, 'novelty': 0.55255264}


# PART 2: ONLINE EVALUATION (codabench)

In [ ]:
# 1. Load full training and official test fold-in
train_full = load_interactions(train=True)
test_in = load_interactions(train=False)

In [ ]:
# 2. New model instance, same lambda
model_cb = EASE(lambda_reg=LAMBDA_REG)

In [ ]:
# 3. train_full is used to fit, test_in defines which users we score
recs_codabench = model_cb.recommend(train_full, test_in, top_k=TOP_K)

In [ ]:
# Quick sanity check
print(recs_codabench.head())

In [ ]:
# 7. Save submission CSV (this writes submission_name.csv in notebooks or pipelines location)
submission_name = f"ease_baseline_lambda_{LAMBDA_REG}"
save_submission(recs_codabench, submission_name)
print(f"Saved Codabench submission as {submission_name}.csv")